# Full Reconstruction Workflow — Classical First Pass

End-to-end demonstration using the `lhcb_velo_toy` package:
1. Configure detector geometry
2. Generate events with configurable physics
3. Build Hamiltonian  
4. Solve classically (replacing quantum solver for first pass)
5. Extract and validate tracks
6. Visualise results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from lhcb_velo_toy import (
    PlaneGeometry,
    StateEventGenerator,
    SimpleHamiltonian,
    EventValidator,
    get_tracks,
    get_segments_from_event,
)

os.makedirs('Plots', exist_ok=True)
print('Imports OK')

Imports OK


---
## 1. Detector Geometry

In [2]:
# 5-layer VELO-like detector
n_layers = 5
dz = 20  # mm spacing

module_ids = list(range(1, n_layers + 1))
lx = [33] * n_layers  # sensor half-width
ly = [33] * n_layers  # sensor half-height
zs = [dz * i for i in range(1, n_layers + 1)]

detector = PlaneGeometry(module_id=module_ids, lx=lx, ly=ly, z=zs)

print(f"Detector: {n_layers} modules")
print(f"z-positions: {zs}")
print(f"Module area: {2*lx[0]} × {2*ly[0]} mm²")

Detector: 5 modules
z-positions: [20, 40, 60, 80, 100]
Module area: 66 × 66 mm²


---
## 2. Event Generation

In [3]:
n_particles = 15
measurement_sigma = 5e-3  # 5 μm
scattering_sigma = 0.2e-3  # 0.2 mrad

gen = StateEventGenerator(
    detector, events=1,
    n_particles=[n_particles],
    measurement_error=measurement_sigma,
    collision_noise=scattering_sigma,
)

gen.generate_random_primary_vertices({'x': 0, 'y': 0, 'z': 0})

particles = [[{'type': 'MIP', 'mass': 0.511, 'q': 1}
               for _ in range(n_particles)]]
gen.generate_particles(particles)

event = gen.generate_complete_events()

print(f"Event summary:")
print(f"  Tracks:  {event.n_tracks}")
print(f"  Hits:    {event.n_hits}")
print(f"  Modules: {event.n_modules}")

Event summary:
  Tracks:  15
  Hits:    75
  Modules: 5


---
## 3. Build Hamiltonian

The penalty Hamiltonian has the form:

$$H(\mathbf{x}) = -\gamma \sum_i x_i + \delta \sum_{i \neq j} P_{ij} x_i x_j$$

- $\gamma$ controls the reward for including a segment
- $\delta$ controls the penalty for incompatible segment pairs  
- $\varepsilon$ is the angular tolerance

In [4]:
epsilon = 1e-7  # angular tolerance
gamma = 2.0     # reward weight (old: alpha)
delta = 1.0     # penalty weight (old: beta)

ham = SimpleHamiltonian(epsilon=epsilon, gamma=gamma, delta=delta)
ham.construct_hamiltonian(event, convolution=False)

n_segments = len(ham.segments)
n_true = sum(1 for s in ham.segments
             if s.hit_start.track_id == s.hit_end.track_id and s.hit_start.track_id >= 0)

print(f"Hamiltonian constructed:")
print(f"  Segments (QUBO variables): {n_segments}")
print(f"  True segments:              {n_true}")
print(f"  False segments:             {n_segments - n_true}")
print(f"  Matrix shape:               {ham.Q.shape}")
print(f"  Matrix density:             {(ham.Q.nnz / ham.Q.shape[0]**2):.4f}")

Hamiltonian constructed:
  Segments (QUBO variables): 900
  True segments:              60
  False segments:             840


AttributeError: 'SimpleHamiltonian' object has no attribute 'Q'

---
## 4. Classical Solution

Using Conjugate Gradient method (`scipy.sparse.linalg.cg`) as the classical solver.  
In a production run this would be replaced by the quantum solver (OneBitHHL / HHLAlgorithm).

In [ ]:
# Solve
solution = ham.solve_classicaly()

# Discretize
threshold = 0.45
binary = (solution > threshold).astype(int)

print(f"Solution statistics:")
print(f"  min={solution.min():.4f}, max={solution.max():.4f}, mean={solution.mean():.4f}")
print(f"  Active segments: {binary.sum()} / {len(binary)}")
print(f"  Expected true:   {n_true}")

# Hamiltonian energy
E = ham.evaluate(binary)
print(f"  H(x_binary) = {E:.4f}")

In [ ]:
# Solution visualisation
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# (a) Continuous solution
colors = ['#492EAB' if s.hit_start.track_id == s.hit_end.track_id and s.hit_start.track_id >= 0
          else '#E74C3C' for s in ham.segments]
axes[0].bar(range(n_segments), solution, color=colors, alpha=0.7, edgecolor='black', lw=0.3)
axes[0].axhline(threshold, color='black', ls='--', lw=2, label=f'threshold={threshold}')
axes[0].set_xlabel('Segment Index')
axes[0].set_ylabel('Activation')
axes[0].set_title('(a) Continuous Solution', fontweight='bold')
axes[0].legend()

# (b) Binary solution
bar_colors = ['#06A77D' if b == 1 else '#ddd' for b in binary]
axes[1].bar(range(n_segments), binary, color=bar_colors, edgecolor='black', lw=0.3)
axes[1].set_xlabel('Segment Index')
axes[1].set_ylabel('Binary Value')
axes[1].set_title('(b) Discretized Binary', fontweight='bold')
axes[1].set_ylim(-0.1, 1.1)

# (c) Confusion breakdown
tp, fp, fn = 0, 0, 0
for s, b in zip(ham.segments, binary):
    is_true = (s.hit_start.track_id == s.hit_end.track_id and s.hit_start.track_id >= 0)
    if b == 1 and is_true: tp += 1
    elif b == 1 and not is_true: fp += 1
    elif b == 0 and is_true: fn += 1
tn = n_segments - tp - fp - fn

labels = ['True +', 'False +', 'False −', 'True −']
sizes = [tp, fp, fn, tn]
colors_pie = ['#06A77D', '#E74C3C', '#F39C12', '#95A5A6']
explode = (0.05, 0.05, 0.05, 0)
axes[2].pie(sizes, labels=labels, colors=colors_pie, autopct='%1.0f%%',
            explode=explode, startangle=90, textprops={'fontsize': 11})
axes[2].set_title('(c) Segment Classification', fontweight='bold')

plt.tight_layout()
plt.savefig('Plots/workflow_solution.jpeg', dpi=300, bbox_inches='tight')
plt.show()
print(f"TP={tp}, FP={fp}, FN={fn}, TN={tn}")

---
## 5. Track Extraction & Validation

In [ ]:
# Extract tracks
reco_tracks = get_tracks(ham, binary, event)

print(f"Reconstructed tracks: {len(reco_tracks)}")
for i, t in enumerate(reco_tracks):
    print(f"  Track {i}: {len(t.hit_ids)} hits")

# Validate
validator = EventValidator(event, reco_tracks)
matches, metrics = validator.match_tracks(purity_min=0.7)

print(f"\n{'='*50}")
print(f"RECONSTRUCTION METRICS")
print(f"{'='*50}")
print(f"  Efficiency:      {metrics['efficiency']:.1%}")
print(f"  Ghost Rate:      {metrics['ghost_rate']:.1%}")
print(f"  Clone Fraction:  {metrics['clone_fraction']:.1%}")
print(f"  Mean Purity:     {metrics['mean_purity']:.3f}")
print(f"  Mean Hit Eff:    {metrics['mean_hit_efficiency']:.3f}")

---
## 6. Event Display

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(22, 6))

# Collect hit data
hit_x = [h.x for h in event.hits]
hit_y = [h.y for h in event.hits]
hit_z = [h.z for h in event.hits]

cmap = plt.cm.tab20
track_ids_set = sorted(set(h.track_id for h in event.hits if h.track_id >= 0))
color_map = {tid: cmap(i % 20) for i, tid in enumerate(track_ids_set)}

# --- (a) True Tracks ---
ax = axes[0]
for tid in track_ids_set:
    t_hits = sorted([h for h in event.hits if h.track_id == tid], key=lambda h: h.z)
    xs = [h.x for h in t_hits]
    zs_ = [h.z for h in t_hits]
    ax.plot(zs_, xs, 'o-', color=color_map[tid], ms=5, lw=1.5)
ax.set_xlabel('z (mm)'); ax.set_ylabel('x (mm)')
ax.set_title('(a) True Tracks', fontweight='bold', fontsize=13)
ax.grid(alpha=0.3)

# --- (b) Reconstructed Tracks ---
ax = axes[1]
reco_cmap = plt.cm.Set2
for i, t in enumerate(reco_tracks):
    r_hits = sorted([event.get_hit_by_id(hid) for hid in t.hit_ids], key=lambda h: h.z)
    xs = [h.x for h in r_hits]
    zs_ = [h.z for h in r_hits]
    ax.plot(zs_, xs, 's-', color=reco_cmap(i % 8), ms=5, lw=1.5)
ax.set_xlabel('z (mm)'); ax.set_ylabel('x (mm)')
ax.set_title('(b) Reconstructed Tracks', fontweight='bold', fontsize=13)
ax.grid(alpha=0.3)

# --- (c) Active Segments ---
ax = axes[2]
ax.scatter(hit_z, hit_x, c='lightgray', s=20, zorder=1)
for seg, b in zip(ham.segments, binary):
    if b == 1:
        is_true = (seg.hit_start.track_id == seg.hit_end.track_id and seg.hit_start.track_id >= 0)
        color = '#06A77D' if is_true else '#E74C3C'
        ax.plot([seg.hit_start.z, seg.hit_end.z],
                [seg.hit_start.x, seg.hit_end.x],
                '-', color=color, lw=1.5, alpha=0.8)
ax.set_xlabel('z (mm)'); ax.set_ylabel('x (mm)')
ax.set_title('(c) Active Segments (green=true)', fontweight='bold', fontsize=13)
ax.grid(alpha=0.3)

plt.suptitle(f'Event Display — {n_particles} tracks, {n_layers} layers',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('Plots/workflow_event_display.jpeg', dpi=300, bbox_inches='tight')
plt.show()

---
## 7. Match Details

In [ ]:
print(f"{'Reco':<6} {'True':<6} {'Purity':<8} {'Hit Eff':<8} {'Status'}")
print('-' * 50)
for m in matches:
    status = 'Ghost' if m.is_ghost else ('Clone' if m.is_clone else 'Matched')
    tid_str = str(m.true_track_id) if m.true_track_id is not None else '—'
    print(f"{m.reco_track_id:<6} {tid_str:<6} {m.purity:<8.3f} {m.hit_efficiency:<8.3f} {status}")

---
## Summary

This notebook demonstrates the complete classical track reconstruction pipeline:

| Step | Method | Status |
|------|--------|--------|
| Geometry | `PlaneGeometry` | New API |
| Generation | `StateEventGenerator` | New API |
| Hamiltonian | `SimpleHamiltonian(gamma, delta)` | New API (renamed from α,β) |
| Solver | `solve_classicaly()` | Classical CG |
| Track Extraction | `get_tracks()` | New API |
| Validation | `EventValidator.match_tracks()` | LHCb-style matching |

For the quantum solver path, replace step 4 with `OneBitHHL` or `HHLAlgorithm` from `lhcb_velo_toy.solvers.quantum`.